<a href="https://colab.research.google.com/github/Adetayo047/Abosedeuk/blob/main/Intent_Recognition_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT RE-WRITTEN

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00


In [2]:
# Reading Data from csvfiles
import pandas as pd

!gdown --id 1OlcvGWReJMuyYQuOZm149vHWwPtlboR6 --output train.csv
!gdown --id 1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w --output valid.csv
!gdown --id 1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF --output test.csv

train = pd.read_csv("train.csv")
valid = pd.read_csv("valid.csv")
test = pd.read_csv("test.csv")

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1OlcvGWReJMuyYQuOZm149vHWwPtlboR6
To: /content/train.csv
100% 799k/799k [00:00<00:00, 42.1MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w
To: /content/valid.csv
100% 43.3k/43.3k [00:00<00:00, 75.8MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1e

In [3]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

# Load the dataset

# Split the data into training and testing sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(
 #   data['text'], data['intent'], test_size=0.2, random_state=42
#)

# Load the BERT tokenizer and encode the texts

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['intent_enc'] = le.fit_transform(train['intent'])
test['intent_enc'] = le.fit_transform(test['intent'])


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(train.text), truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(list(test.text), truncation=True, padding=True, return_tensors='tf')

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train.intent_enc))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test.intent_enc))

# Fine-tune the pre-trained BERT model for intent recognition
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=train['intent_enc'].nunique())
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
model.fit(train_dataset.shuffle(len(train.text)).batch(32), epochs=3, batch_size=32)

# Evaluate the model on the test set
results = model.evaluate(test_dataset.batch(32))
print("Test accuracy:", results[1])


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
409/409 [==============================] - 160s 270ms/step - loss: 0.3130 - accuracy: 0.9304
Epoch 2/3
409/409 [==============================] - 106s 260ms/step - loss: 0.0422 - accuracy: 0.9908
Epoch 3/3
22/22 [==============================] - 4s 75ms/step - loss: 0.0635 - accuracy: 0.9800
Test accuracy: 0.9800000190734863


In [4]:
model.save_pretrained('bertmodelintent')

# Optionally, you can save the tokenizer as well
tokenizer.save_pretrained('berttokenizer')

('berttokenizer/tokenizer_config.json',
 'berttokenizer/special_tokens_map.json',
 'berttokenizer/vocab.txt',
 'berttokenizer/added_tokens.json')

In [5]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load the model from the saved directory
model = TFBertForSequenceClassification.from_pretrained('bertmodelintent')

# Optionally, load the tokenizer as well
tokenizer = BertTokenizer.from_pretrained('berttokenizer')

# Function to predict intent from a given text
def predict_intent(text):
    # Tokenize the text
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors='tf')

    # Get the model's prediction
    predictions = model(inputs)[0]
    predicted_label_id = tf.argmax(predictions, axis=1).numpy()[0]

    return le.inverse_transform([predicted_label_id])[0]

# Example usage


Some layers from the model checkpoint at bertmodelintent were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at bertmodelintent.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [6]:
text_to_predict = "Search for nearby place to eat dinner"
predicted_intent = predict_intent(text_to_predict)
print("Predicted intent:", predicted_intent)

Predicted intent: BookRestaurant


In [ ]:
test